# Parallel Execution

# Creating image paths for dataset




In [ ]:
datadir='/cotent/PetImages'
p_training_data=[]

Dog_categories=['Dog']
for category in Dog_categories:
  path=os.path.join(datadir,category) #path to cat or dog categories
  imagePaths1=[os.path.join(path,img) for img in os.listdir(path)]

Cat_categories=['Cat']
for category in Cat_categories:
  path=os.path.join(datadir,category) #path to cat or dog categories
  imagePaths2=[os.path.join(path,img) for img in os.listdir(path)]



# Creating Dog dataset parallely

In [ ]:
classifier=0

In [ ]:
from multiprocessing import Pool
import os
import cv2
import time

p_dog_training_data=[]
start=time.time()

def import_training_set(args):
  c=0
  global classifier
  classifier+=1
  index,imagePath,new_image_size=args
  #Reads the image
  image=cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)

try:
  input_vector=cv2.resize(image,new_image_size)
  p_dog_training_data.append([input_vector,0])
  return p_dog_training_data

except Exception as e:
  c+=1

new_image_size=120,120
number_of_parallel_processes=2

ans1=Pool(number_of_parallel_processes).map(import_training_set,[(i,img,new_image_size) for i,img in enumerate(imagePaths1)])

end=time.time()

p_dog_dataset_creating_time=end-start

In [ ]:
print(p_dog_dataset_creating_time)

# Creating Cat dataset parallely

In [ ]:
from multiprocessing import Pool
import os
import cv2
import time

p_cat_training_data=[]
start=time.time()

def import_training_set(args):
  c=0
  global classifier
  classifier+=1
  index,imagePath,new_image_size=args
  #Reads the image
  image=cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)

try:
  input_vector=cv2.resize(image,new_image_size)
  p_cat_training_data.append([input_vector,1])
  return p_cat_training_data

except Exception as e:
  c+=1

new_image_size=120,120
number_of_parallel_processes=2

ans2=Pool(number_of_parallel_processes).map(import_training_set,[(i,img,new_image_size) for i,img in enumerate(imagePaths2)])

end=time.time()

p_cat_dataset_creating_time=end-start

In [ ]:
print(p_cat_dataset_creating_time)

Total time for creating dataset parallely

In [ ]:
p_total_dataset_creating_time=p_dog_dataset_creating_time+p_cat_dataset_creating_time
print(p_total_dataset_creating_time)

# Training the dataset (model)

In [ ]:
import tenserflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D

import pickle
import time

X1 = pickle.load(open("X1.pickle","rb"))
y1 = pickle.load(open("y1.pickle","rb"))

X1 = X1/255.0

start=time.time()

model=Sequential()

model.add(Conv2D(64,(3,3),input_shape=X1.shape[1:]))

model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,batch_size=20,epochs=10,validation_split=0.1)

end=time.time()
p_training_time = end-start




Total time for training the data

In [ ]:
print(p_training_time)

# Total Parallel Execution Time

In [ ]:
total_parallel_execution_time=p_total_dataset_creating_time+p_training_time
print(total_parallel_execution_time)

# Making a prediction

In [ ]:
import cv2
import tenserflow as tf

categories=['Dog','Cat']

def prepare(filepath):
  image_size=120
  a=cv2.imread(filepath)
  a1=cv2.resize(a,(img_size,img_size))
  return reshape(-1,img_size,img_size,1)

prediction=model.predict([prepare('/cotent/dog1.png')])
print(categories[int(prediction[0][0])])